In [1]:
from création_data import param_Dataset
import fonction_model as f
from IPython.display import display, Audio
from scipy.io.wavfile import write
import pickle

In [2]:
Fe = 40000

In [3]:
Fichier = "1ier_essai/composite/acier/"

In [6]:
N_sample,param_dataset,xinB = param_Dataset(N_sample=10,
                                            article_C = False,
                                            acier_1C = False,
                                            acier_2C = True, 
                                            medium_1T = True, 
                                            medium_2T= False, 
                                            metal_T = False,
                                            plexi_T = False)

0.0031756795199999995


In [7]:
from IPython.display import Audio, display
T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly = param_dataset[0]
F = f.Main(T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly,xinB,Fe)

/Users/armandschwarz/Documents/GitHub/ATIAM-PAM-AnalyseInstruments/Data/fonction_model.py:301: RuntimeWarning: invalid value encountered in matmul
  q_dd_temp[:,i+1] = W @  q_pour_f[:,i+1]
/Users/armandschwarz/Documents/GitHub/ATIAM-PAM-AnalyseInstruments/Data/fonction_model.py:307: RuntimeWarning: invalid value encountered in matmul
  F_c = Z @ q_pour_f


In [ ]:
display(Audio(F,rate=Fe))

In [ ]:
for i in range(N_sample):
    T,rho_l,L,E_corde,I,h,nu,E_table,rhoT,Lx,Ly = param_dataset[i]
    F  = f.Main(T,rho_l,L,E_corde,I,h,nu,E_table,rhoT,Lx,Ly,Fe)
    write(Fichier + str(i) + '.wav',rate=Fe ,data = F[-1,:] / max(F[-1,:]))


In [ ]:
dico = {}
for i in range(N_sample):
    kle = str(i) + ".wav"
    dico[kle] = {}
    dico[kle]['T_corde'] = param_dataset[i][0]
    dico[kle]['rho_l_corde'] = param_dataset[i][1]
    dico[kle]['L_corde'] = param_dataset[i][2]
    dico[kle]['E_corde'] = param_dataset[i][3]
    dico[kle]['I_corde'] = param_dataset[i][4]
    dico[kle]['h_table'] = param_dataset[i][5]
    dico[kle]['nu_table'] = param_dataset[i][6]
    dico[kle]['E_table'] = param_dataset[i][7]
    dico[kle]['rho_table'] = param_dataset[i][8]
    dico[kle]['Lx_table'] = param_dataset[i][9]
    dico[kle]['Ly_table'] = param_dataset[i][10]

with open(Fichier + 'parametres.pickle', 'wb') as f:
    pickle.dump(dico, f)